<h2><font color="#004D7F" size=5>Módulo 2: Bootstrap Aggregation</font></h2>



<h1><font color="#004D7F" size=6>2. Programación Random Forest </font></h1>

<br><br>
<div style="text-align: right">
<font color="#004D7F" size=3>Manuel Castillo-Cara</font><br>
<font color="#004D7F" size=3>Aprendizaje Automático II</font><br>
<font color="#004D7F" size=3>Universidad Nacional de Educación a Distancia</font>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>


* [1. Introducción](#section1)
   * [1.1. Algoritmo Random Forest](#section11)
   * [1.2. Dataset](#section12)
   * [1.3. Tutorial](#section13)
* [2. Calcular las divisiones](#section2)
* [3. Caso de estudio: dataset Sonar](#section3)
   * [3.1. Resultados](#section31)
* [Ejercicios](#sectionEj)




---

<a id="section0"></a>
# <font color="#004D7F">0. Contexto</font>

Los árboles de decisión pueden sufrir de gran varianza, lo que hace que sus resultados sean frágiles para los datos de entrenamiento específicos utilizados. La creación de múltiples modelos a partir de muestras de sus datos de entrenamiento, lo que se denomina bagging, puede reducir esta varianza, pero los árboles están altamente correlacionados.

Random Forest es una extensión bagging que, además de construir árboles basándose en múltiples muestras de sus datos de entrenamiento, también restringe las características que se pueden usar para construir los árboles, lo que obliga a los árboles a ser diferentes. Esto, a su vez, puede aumentar el rendimiento. En este tutorial, descubrirá cómo implementar el algoritmo Random Forest desde cero en Python. Después de completar este tutorial, sabrá:
- La diferencia entre árboles de decisión en bagging y el algoritmo Random Forest.
- Cómo construir árboles de decisión en bagging con más variación.
- Cómo aplicar el algoritmo de bosque aleatorio a un problema de modelado predictivo.

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section1"></a>
# <font color="#004D7F"> 1. Introducción</font>

Esta sección proporciona una breve descripción de Bootstrap Aggregation y el conjunto de datos de Sonar que se utilizarán en este tutorial.

<a id="section11"></a>
## <font color="#004D7F"> 1.1. Algoritmo Random Forest</font>

Los árboles de decisión implican la selección codiciosa del mejor punto de división del conjunto de datos en cada paso. Este algoritmo hace que los árboles de decisión sean susceptibles a una gran variación si no se podan. Esta alta varianza se puede aprovechar y reducir creando múltiples árboles con diferentes muestras del conjunto de datos de entrenamiento (diferentes vistas del problema) y combinando sus predicciones. Este enfoque se llama bootstrap aggregation o bagging para abreviar.

Una limitación de bagging es que se utiliza el mismo algoritmo codicioso para crear cada árbol, lo que significa que es probable que se elijan puntos de división iguales o muy similares en cada árbol, lo que hace que los diferentes árboles sean muy similares (los árboles estarán correlacionados). Esto, a su vez, hace que sus predicciones sean similares, mitigando la variación buscada originalmente.

Podemos forzar que los árboles de decisión sean diferentes limitando las características (columnas) que el
algoritmo codicioso puede evaluar en cada punto de división al crear el árbol. Esto se llama algoritmo de Random Forest. 
- Al igual que el bagging, se toman varias muestras del conjunto de datos de entrenamiento y se
entrena un árbol diferente en cada una. 
- La diferencia es que en cada punto se realiza una división de los datos y se agrega al árbol; solo se puede considerar un subconjunto fijo de atributos. 
- Para problemas de clasificación, el tipo de problemas que veremos en este tutorial, la cantidad de atributos que se considerarán para la división se limita a la raíz cuadrada de la cantidad de características de entrada.
$$
       num\_features\_for\_split = \sqrt{total\_input\_features}
$$

El resultado de este pequeño cambio son árboles que son más diferentes entre sí (no correlacionados), lo que da como resultado predicciones más diversas y una predicción combinada que a menudo tiene mejor rendimiento que un solo árbol o el bagging por sí solo.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: En Scikit-learn podemos ver los diferentes parámetros que tiene [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) y [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)
</div>

<a id="section12"></a>
## <font color="#004D7F"> 1.2. Dataset</font>

En este tutorial usaremos el conjunto de datos Sonar. 
- Este conjunto de datos implica la discriminación entre minas y rocas. 
- El desempeño de referencia sobre el problema es aproximadamente del 53%. 

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Más información sobre el dataset [Sonar](https://archive.ics.uci.edu/dataset/151/connectionist+bench+sonar+mines+vs+rocks)
</div>

<a id="section13"></a>
## <font color="#004D7F"> 1.3. Tutorial</font>

Este tutorial se divide en 2 partes:
- Cálculo de divisiones.
- Caso de estudio Sonar.
Estos pasos proporcionan la base que necesita para implementar y aplicar Random Forest a sus propios problemas de modelado predictivo.

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section2"></a> 
# <font color="#004D7F"> 2. Calcular divisiones</font>

En un árbol de decisión, los puntos de división se eligen encontrando el atributo y el valor de ese atributo que resulta en el costo más bajo. 
- Para problemas de clasificación, esta función de costos suele ser el índice de Gini que calcula la pureza de los grupos de datos creados por el punto de división. 
- Un índice de Gini de 0 es una pureza perfecta donde los valores de clase están perfectamente separados en dos grupos, en el caso de un problema de clasificación de dos clases.
- Encontrar el mejor punto de división en un árbol de decisión implica evaluar el costo de cada valor en el conjunto de datos de entrenamiento para cada variable de entrada. 

Para Bagging y bosque aleatorio, este procedimiento se ejecuta sobre una muestra del conjunto de datos de entrenamiento, realizado con reemplazo. El muestreo con reemplazo significa que se puede elegir la misma fila y agregarla a la muestra más de una vez.

Podemos actualizar este procedimiento para Random Forest. 
- En lugar de enumerar todos los valores de los atributos de entrada en busca de la división con el costo más bajo, podemos crear una muestra de los atributos de entrada a considerar. 
- Esta muestra de atributos de entrada se puede elegir aleatoriamente y sin reemplazo, lo que significa que cada atributo de entrada solo debe considerarse una vez cuando se busca el punto de división con el costo más bajo.

Veamos código:
- A continuación se muestra el nombre de la función `get_split()` que implementa este procedimiento. 
- Se necesita un conjunto de datos y una cantidad fija de características de entrada para evaluar como argumentos de entrada, donde el conjunto de datos puede ser una muestra del conjunto de datos de entrenamiento real. 
- La función auxiliar `test_split()` se usa para dividir el conjunto de datos por un punto de división candidato y
- El `gini_index()` se usa para evaluar el costo de una división determinada por los grupos de filas creados.
- Podemos ver que se crea una lista de características seleccionando aleatoriamente índices de características y agregándolos a una lista (llamadas `features`), 
- Luego esta lista de características se enumera y los valores específicos en el conjunto de datos de entrenamiento se evalúan como puntos de división.

In [1]:
# Select the best split point for a dataset
def get_split(dataset, n_features):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	features = list()
	while len(features) < n_features:
		index = randrange(len(dataset[0])-1)
		if index not in features:
			features.append(index)
	for index in features:
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

Ahora que sabemos cómo se puede modificar un algoritmo de árbol de decisión para usarlo con el algoritmo Random Forest, podemos unirlo con una implementación de bagging y aplicarlo a un conjunto de datos del mundo real.

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section3"></a> 
# <font color="#004D7F"> 3. Caso de estudio: dataset Sonar</font>

En esta sección, aplicaremos el algoritmo Random Forest al conjunto de datos de Sonar. 
1. Primero se carga el conjunto de datos, los valores de cadena se convierten a numéricos y la columna de salida se convierte de cadenas a valores enteros de 0 y 1. 
    - Esto se logra con las funciones auxiliares `load_csv()`, `str_column_to_float()` y `str_column_to_int()` para cargar y preparar el conjunto de datos.
2. Utilizaremos _k_-fold para estimar el rendimiento del modelo aprendido en datos invisibles. 
    - Esto significa que construiremos y evaluaremos _k_ modelos y estimaremos el rendimiento como el error medio del modelo. 
    - El accuracy de la clasificación se utilizará para evaluar cada modelo. 
    - Estos comportamientos se proporcionan en las funciones `cross_valiation_split()`, `accuracy_metric()` y `evaluate_algorithm()`.
3. También usaremos una implementación del algoritmo CART
adaptado para bagging y las funciones auxiliares del Capítulo 11 , incluyendo: 
    - `test split()` para dividir un conjunto de datos en grupos, 
    - `gini_index()` para evaluar un punto de división, 
    - `get_split()` para encontrar un punto de división óptimo, 
    - `to_terminal()`, `split()` y `build_tree()` se usan para crear un único árbol de decisión, 
    - `predict()` para hacer una predicción con un árbol de decisión y
    - `subsample()` descrita en el paso anterior para hacer una submuestra del conjunto de datos de entrenamiento.
    - `bagging_predict()` para hacer una predicción con una lista de árboles de decisión.
5. Se desarrolla una nueva función llamada `random_forest()` que primero crea una lista de árboles de decisión a partir de submuestras del conjunto de datos de entrenamiento y luego los utiliza para hacer predicciones.
    - Como dijimos anteriormente, la diferencia clave entre Random Forest y los árboles de decisión en bagged es el único pequeño cambio en la forma en que se crean los árboles aquí en la función `get_split()`.

El ejemplo completo se enumera a continuación.

In [3]:
# Random Forest Algorithm on Sonar Dataset
from random import seed
from random import randrange
from csv import reader
from math import sqrt

# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for _ in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

# Create a terminal node value
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left, n_features)
		split(node['left'], max_depth, min_size, n_features, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right, n_features)
		split(node['right'], max_depth, min_size, n_features, depth+1)

# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
	root = get_split(train, n_features)
	split(root, max_depth, min_size, n_features, 1)
	return root

# Make a prediction with a decision tree
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
	sample = list()
	n_sample = round(len(dataset) * ratio)
	while len(sample) < n_sample:
		index = randrange(len(dataset))
		sample.append(dataset[index])
	return sample

# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
	predictions = [predict(tree, row) for tree in trees]
	return max(set(predictions), key=predictions.count)

# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
	trees = list()
	for _ in range(n_trees):
		sample = subsample(train, sample_size)
		tree = build_tree(sample, max_depth, min_size, n_features)
		trees.append(tree)
	predictions = [bagging_predict(trees, row) for row in test]
	return(predictions)

# Test the random forest algorithm on sonar dataset
seed(2)
# load and prepare data
filename = 'data/sonar.all-data.csv'
dataset = load_csv(filename)
# convert string attributes to integers
for i in range(0, len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [1, 5, 10]:
	scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
	print('Trees: %d' % n_trees)
	print('Scores: %s' % scores)
	print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Trees: 1
Scores: [56.09756097560976, 63.41463414634146, 60.97560975609756, 58.536585365853654, 73.17073170731707]
Mean Accuracy: 62.439%
Trees: 5
Scores: [70.73170731707317, 58.536585365853654, 85.36585365853658, 75.60975609756098, 63.41463414634146]
Mean Accuracy: 70.732%
Trees: 10
Scores: [75.60975609756098, 80.48780487804879, 92.6829268292683, 73.17073170731707, 70.73170731707317]
Mean Accuracy: 78.537%


<a id="section31"></a> 
## <font color="#004D7F"> 3.1. Resultados</font>

- Se utilizó un valor _k_ de 5 para la validación cruzada, dando a cada pliegue $\frac{208}{5} = 41.6$ o justo 40 registros que se evaluarán en cada iteración.
- Se construyeron árboles con una profundidad máxima de 10 y un número mínimo de filas de entrenamiento en cada nodo de 1. 
- Se crearon muestras del conjunto de datos de entrenamiento con el mismo tamaño que el conjunto de datos original, que es una expectativa predeterminada para el algoritmo Random Forest.
- El número de características consideradas en cada punto de división se estableció en $\sqrt{60} = 7,74$ redondeado a 7 características. 
- Se evaluó un conjunto de 3 números diferentes de árboles para comparar, lo que muestra la habilidad cada vez mayor a medida que se agregan más árboles. 
- Al ejecutar el ejemplo se imprimen las puntuaciones de cada pliegue y la puntuación media de cada configuración.

---
<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="sectionEj"></a>
<h3><font color="#004D7F" size=6> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> Ejercicios</font></h3>

Se proponen las siguientes actividades para consolidar el aprendizaje.

# <font color="#004D7F" size=5>Ejercicio 1</font>
__Ajuste de algoritmos__. La configuración utilizada en el tutorial se encontró con un poco de prueba y error pero no estaba optimizada. Experimente con una mayor cantidad de árboles, diferentes cantidades de características e incluso diferentes configuraciones de árboles para mejorar el rendimiento.

# <font color="#004D7F" size=5>Ejercicio 2</font>
__Más problemas__. Aplicar la técnica a otros problemas de clasificación e incluso adaptarla para regresión con una nueva función de costos y un nuevo método para combinar las predicciones de los árboles.

---

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>